In [1]:
from pylab import *

In [2]:
def I_gamma(Egamma, theta, phi, gas_model, hadronic_model, cr_model):
    
    # Egamma - Gamma-ray energy [GeV]
    # theta - Latitude angle [degrees]
    # phi - Longitude angle [degrees]
    # gas_model - Options: GALPROP
    # hadronic_model - Options: Geant4, Pythia8, SIBYLL, QGSJET
    # cr_model = Options: Dembinski, Lipari
    
    indexE = np.where(np.logical_and(E_Dembinski>=Egamma, E_Dembinski>=Egamma))
    Ep = E_Dembinski[indexE]
    
    # FIRST INTEGRAL
    
    # Cross-section model
    if hadronic_model == 'Geant4':
        dsigmadE = diffcrossGeant4(Ep, Egamma) * 1e-27
    if hadronic_model == 'Pythia8':
        dsigmadE = diffcrossPythia8(Ep, Egamma) * 1e-27
    if hadronic_model == 'SIBYLL':
        dsigmadE = diffcrossSIBYLL(Ep, Egamma) * 1e-27
    if hadronic_model == 'QGSJET':
        dsigmadE = diffcrossQGSJET(Ep, Egamma) * 1e-27
        
    # Cosmic rays model
    if cr_model == 'Dembinski':
        flux_CR = phi_total_Dembinski[indexE] / 1e4
    if cr_model == 'Lipari':
        flux_CR = Phi_total_nucleon(Ep) / 1e4
    if cr_model == 'Prevotat':
        flux_CR = I_total(Ep) / 1e4
        
    integral_E = scipy.integrate.simpson(dsigmadE * flux_CR, Ep)
        
    # SECOND INTEGRAL
    
    # Gas model
    los = np.arange(0, 50, 0.1) * 3.08567758128e21 # in cm
    if gas_model == 'GALPROP':
        n_gas = n_GALPROP(los, theta, phi)
        
    # Optical depth
    tau = vec_tauABSiso(Egamma, los)
    
    intensity = scipy.integrate.simpson(n_gas * np.exp(-tau) * integral_E, los) * np.sin(math.radians(theta))
    
    return intensity #in (GeV cm^2 s sr)^-1
vec_I_gamma = np.vectorize(I_gamma)